In [1]:
import os

In [2]:
#os.listdir()

In [3]:
import logging
logging.root.handlers = []  # Jupyter messes up logging so needs a reset
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from smart_open import smart_open

import pandas as pd
import numpy as np
from numpy import random

import seaborn as sns
import matplotlib.pyplot as plt

import re

import gensim
import nltk

import sklearn
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from sklearn.neighbors import KNeighborsClassifier
from sklearn import linear_model
from nltk.corpus import stopwords



2023-12-12 08:48:45,492 : INFO : Note: NumExpr detected 20 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-12-12 08:48:45,493 : INFO : NumExpr defaulting to 8 threads.


In [4]:
sns.set()

In [5]:
!pip install voila
!jupyter nbextension enable --py widgetsnbextension --sys-prefix
!jupyter nbextension enable --py voila --sys-prefix
!jupyter serverextension enable --py voila --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbclassic nbconvert notebook qtconsole run script
server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.
usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runti

# *Analyse de texte et d'images*

Problématique : L'entreprise Avis restau souhaite améliorer sa plateforme avec une nouvelle fonctionnalité de collaboration. Les utilisateurs pourront par exemple poster des avis et des photos sur leur restaurant préféré. Cette initiative devra êrùettre à l’entreprise de mieux comprendre les avis postés par les utilisateurs.

Méthodologie : Il est demandé de proposer un outil qui permette de faire une analyse de faisabilité pour la mise en place de cet outil à partir de trois grands groupes d'éléments: 
1. les avis des utilisateurs afin de comprendre, le cas échéant, des éventuelles raisons d'insatisfaction,
2. des photos postées par les utilisateurs directement sur le site et,
3. un travail de récupération de nouvelles données sur les restaurant afin de vérifier que l'outil est assez robuste pour l'envoyer en production.

Méthodologie technique :

1. Analyse des avis des clients

> - Préparation et filtrage des données ne gardant que les commentaires négatifs
> - Analyse (bigrammes, trigrammes et pondération)
> - Identification des sujets les plus saillants (LDA, T-sne et explicabilité)
> - Apercu de l'analyse de sentiments (TextBlob)

2. Analyse des images postés par les clients
> - Préparation des données (Création des features)
> - Analyse par une méthode non-supérvisée (ACP, clustering, T-SNE, Kmeans)
> - SIFT
> - Modèle VGG16 (Transfer learning)
> - Approche ImageDatagenerator avec data augmentation
> - Approche sans data augmentation
> - Approche avec data augmentation intégrée au modèle

3. Récupération des données et création d'un fichier csv pour faciliter leur utilisation


# 1. Analyse des avis des clients (travail sur le texte)

In [7]:
df = pd.read_csv("df_text.csv", sep=",")

In [8]:
df.head()

,business_id,text,categories
0,WUGRmrfSzQCyeG6mYAM-vw,The food is just average and very overpriced a...,"Restaurants, Latin American, Mexican, Tacos"
1,3wogQmKw729tKKWaVbZvbg,I paid $20 for the gator plate which was the s...,"Herbs & Spices, Seafood, Restaurants, Cajun/Cr..."
2,DEh5BPKE8LHm4PUsH_E1ig,I really like the staff behind the counter and...,"Italian, Salad, Pizza, Restaurants"
3,Oj-XIt4Xj6NSsHzlSt-PsA,This is not Chinese food.\n\nIt's like aliens ...,"Restaurants, Gluten-Free, Asian Fusion, Chines..."
4,tfkE9AOK5zmiJ9nX0jo5tg,Definitely not like it used to be. I ordered t...,"Restaurants, Burgers, American (Traditional)"


In [9]:
df.shape

(100000, 3)

In [10]:
df = df[['text']]

In [11]:
df.shape

(100000, 1)

In [12]:
df.head()

,text
0,The food is just average and very overpriced a...
1,I paid $20 for the gator plate which was the s...
2,I really like the staff behind the counter and...
3,This is not Chinese food.\n\nIt's like aliens ...
4,Definitely not like it used to be. I ordered t...


# Text processing

#### Extraire une fraction du df pour travailler plus facilement avec

In [13]:
# Extraction d'une fraction pour travailler + facilement
df_1 = df
#df_1 = df.sample(frac=0.2)

In [14]:
df_1.shape

(100000, 1)

## 1. Travail sur les documents disponibles

### Enlèvement de la ponctuation

In [15]:
import nltk
import string

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('words')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### 1. Initialisation des dependances nltk

In [16]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

##### Définition d'une fonction de preprocessing : tokenization, lemmatisation, minuscules, filtrage alphabétique, enlèvement des adverbes

In [18]:
! pip install langdetect

### 2. Suppression de ponctuation

2.1.  Fonction de cleaning numéro 1 : fonction pour enlèver la ponctuation

In [19]:
import pandas as pd
import nltk

def supprimer_ponctuation(df_1):
    # Initialize an empty list to store preprocessed text
    preprocess_list = []

    # Iterate through each text entry in the 'Text' column
    for text in df_1['text']:
        # Create a tokenizer object
        tokenizer = nltk.RegexpTokenizer(r"\w+")

        # Tokenize the text, removing punctuation
        tokens = tokenizer.tokenize(text)

        # Join the tokens back into a preprocessed text string
        preprocessed_text = " ".join(tokens)

        # Append the preprocessed text to the list
        preprocess_list.append(preprocessed_text)

    # Assign the preprocess_list to the 'Text' column of the DataFrame
    df_1['text'] = preprocess_list

    return df_1


In [20]:
df_1.head()

,text
0,The food is just average and very overpriced a...
1,I paid $20 for the gator plate which was the s...
2,I really like the staff behind the counter and...
3,This is not Chinese food.\n\nIt's like aliens ...
4,Definitely not like it used to be. I ordered t...


Appel de la fonction qui enlève la pontuation

In [21]:
# Call the supprimer_ponctuation function to preprocess the text
df_1 = supprimer_ponctuation(df_1)

df_1.head()

,text
0,The food is just average and very overpriced a...
1,I paid 20 for the gator plate which was the sa...
2,I really like the staff behind the counter and...
3,This is not Chinese food It s like aliens land...
4,Definitely not like it used to be I ordered th...


In [22]:
# Vérification de l'opération
#index_to_find = 27804

# loc pour trouver l'index
#row = df_1.loc[index_to_find]

# afficahge de la ligne avec row
#print(row)


### 3. Fonction de cleaning numéro 2 : fonction de pré-processing

- Tokenisation
- Lower-case
- Filtrage par langue pour ne garder que les mots en anglais
- Enlèvement des adverbes
- Enlèvement de stopwords
- Lémmatisation


In [23]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('words')

stopwords = nltk.corpus.stopwords.words('english')
words = set(nltk.corpus.words.words())
lemmatizer = WordNetLemmatizer()

def Preprocess_listofSentence(listofSentence):
    preprocess_list = []
    for sentence in listofSentence:
        sentence_lower = sentence.lower()  # Convertir les mots en minuscules
        english_words = set(nltk.corpus.words.words())  # Ne garder que les mots qui sont en anglais
        sentence_clean = " ".join(w for w in nltk.wordpunct_tokenize(sentence_lower) if w in english_words or not w.isalpha())

        tokenize_sentence = word_tokenize(sentence_clean)

        # Part-of-speech tagging to identify adverbs
        pos_tags = nltk.pos_tag(tokenize_sentence)
        words_w_adverbs = [word for word, pos in pos_tags if pos not in ('RB', 'RBR', 'RBS', 'JJR', 'JJS')]

        words_w_stopwords = [word for word in words_w_adverbs if word not in stopwords]

        # Filtrer les mots qui ont moins de 3 lettres
        words_filtered = [word for word in words_w_stopwords if len(word) >= 3]

        words_lemmatize = (lemmatizer.lemmatize(w) for w in words_filtered)

        sentence_clean = ' '.join(w for w in words_lemmatize)

        preprocess_list.append(sentence_clean)

    return preprocess_list


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


### Appel de la fonction de cleaning n.2

In [ ]:
preprocess_list = Preprocess_listofSentence(df_1['text'])

In [ ]:
#Vérification de l'opération
type(preprocess_list)

In [ ]:
import pandas as pd

# Créer un DataFrame avec deux colonnes : une pour le texte original et une pour le texte prétraité
df_1 = pd.DataFrame({'Original_Text': df_1['text'], 'Processed_Text': preprocess_list})

# Augmentez la largeur d'affichage pour voir les colonnes côte à côte
pd.set_option('display.width', 1000)

# Afficher les deux colonnes côte à côte
print(df_1)


In [ ]:
df_1.shape

In [ ]:
#application de la fonctionalité len() sur text avec une lambda sur le text pour voir la longueur de chaque entrée
df_1.Original_Text.apply(lambda i : len(i)) #on obtient un vecteur indiquant la longueur

In [ ]:
#ajout d'une colonne pour voir la logueur du commentaire
df_1["_len_txt_orig"] = df_1.Original_Text.apply(lambda i : len(i))
df_1.head()


In [ ]:
#application de la fonctionalité len() sur text avec une lambda sur le text pour voir la longueur de chaque entrée
#df_1.Processed_Text.apply(lambda i : len(i)) #on obtien un vecteur indiquant la longueur
#ajout d'un colonne pour voir la logueur du commentaire
df_1["_len_txt_proc"] = df_1.Processed_Text.apply(lambda i : len(i))
df_1.head()

In [ ]:
# Classement pour analyse simple de la longueur des commentaires par ordre croissant
df_1.sort_values("_len_txt_orig").head(10)

In [ ]:
# Classement pour analyse simple de la longueur des commentaires par ordre décroissant
df_1.sort_values("_len_txt_orig").tail(10)

In [ ]:
df_1.shape

##### Analyse des mots fréquents qui ne sont pas informatifs de l'insatisfaction des clients

In [ ]:
# Convertir les valeurs de la colonne "Processed_Text" en chaînes de caractères avant la construction d'un "raw_corpus"
df_1['Processed_Text'] = df_1['Processed_Text'].astype(str)

#join all corpus
raw_corpus = "".join(df_1.Processed_Text.values)
raw_corpus[:1_000]


In [ ]:
import nltk
from nltk import FreqDist

# extraction des tokens dans une variable pour continuer le nettoyage
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Calculer la fréquence des mots
word_freq = FreqDist(corpus_tokens)

# Afficher les mots les plus fréquents (par exemple, les 10 premiers)
top_words = word_freq.most_common(10)
print(top_words)



In [ ]:
import nltk
from nltk import FreqDist

# extraction des tokens dans une variable pour continuer le nettoyage
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Calculer la fréquence des mots
word_freq = FreqDist(corpus_tokens)

# Afficher les mots les plus fréquents (par exemple, les 50 premiers)
top_words = word_freq.most_common(100)
print(top_words)



Comptage des tokens pour contrôle avant l'opération d'élimination des mots le splus courants peu significatifs

In [ ]:
#longuer de la liste de tokens
len(corpus_tokens)

In [ ]:
#longuer de la liste de tokens uniques
len(set(corpus_tokens))

##### Elimination des mots fréquents non-informatifs de l'insatisfaction des clients

In [ ]:
import nltk
from nltk import FreqDist

# Extraction des tokens dans une variable pour continuer le nettoyage
df_1_processed = nltk.word_tokenize(" ".join(df_1['Processed_Text']))

# Calculer la fréquence des mots
word_freq = FreqDist(df_1_processed)

# Afficher les mots les plus fréquents (par exemple, les 100 premiers)
top_words = [word for word, freq in word_freq.most_common(100)]

# Liste de mots à conserver
words_to_keep = ['bad', 'another', 'worst', 'rude', "wasn't", 'horrible', 'cold', 'last', 'customer', 'service', 'wasnt', 'worst', 'terrible']

# Liste pour stocker les mots qui ne font pas partie de top_words mais qui sont dans words_to_keep
filtered_tokens = [word for word in df_1_processed if word not in top_words or word in words_to_keep]

# Rejoindre les mots filtrés pour former un nouveau texte
filtered_corpus = " ".join(filtered_tokens)

# Les mots de la liste words_to_keep seront conservés, même s'ils sont courants
corpus_tokens = nltk.word_tokenize(filtered_corpus)

# Affichez les 10 premiers mots du corpus_tokens
print(corpus_tokens[:10])


In [ ]:
from nltk import FreqDist
import pandas as pd

# Calculer la fréquence des mots dans le corpus nettoyé
word_freq = FreqDist(df_1_processed)

# Calculer le nombre total de documents
total_documents = len(df_1)

# Définir un seuil pour la fréquence minimale (20%)
min_frequency = 0.2 * total_documents

# Sélectionner les mots qui apparaissent dans au moins 20% des textes
common_words = [word for word, freq in word_freq.items() if freq >= min_frequency]

# Liste de mots à conserver (en plus des mots communs)
words_to_keep = ['bad', 'another', 'worst', 'rude', "wasn't", 'horrible', 'cold', 'last', 'customer', 'service', 'wasnt', 'worst', 'terrible']

# Liste pour stocker les mots qui ne font pas partie de common_words mais qui sont dans words_to_keep
filtered_tokens = [word for word in df_1_processed if word not in common_words or word in words_to_keep]

# Rejoindre les mots filtrés pour former un nouveau texte
filtered_corpus = " ".join(filtered_tokens)

# Les mots de la liste words_to_keep seront conservés, même s'ils sont courants
corpus_tokens = nltk.word_tokenize(filtered_corpus)

# Affichez les 10 premiers mots du corpus_tokens
print(corpus_tokens[:10])


In [ ]:
df_1.shape

In [ ]:
print(corpus_tokens[:10])

In [ ]:
from nltk import FreqDist

# Tokenize le corpus nettoyé
corpus_tokens = nltk.word_tokenize(filtered_corpus)

# Calculer la fréquence des mots dans le corpus nettoyé
word_freq_after_cleaning = FreqDist(corpus_tokens)

# Afficher les 10 mots les plus fréquents
top_words_after_cleaning = word_freq_after_cleaning.most_common(20)

# Afficher les résultats
print(top_words_after_cleaning)


In [ ]:
# Classement pour analyse simple de la longueur des commentaires par ordre croissant
df_1.sort_values("_len_txt_orig").tail(10)

In [ ]:
len(raw_corpus)

Comptage des mots

In [ ]:
raw_corpus[:1_000]

In [ ]:
len(raw_corpus)

In [ ]:
#longuer de la liste de tokens
len(corpus_tokens)

In [ ]:
#longuer de la liste de tokens uniques
len(set(corpus_tokens))

In [ ]:
#value counts
corpus_tokens_count = pd.Series(corpus_tokens).value_counts()
corpus_tokens_count

In [ ]:
corpus_tokens_count.describe()

In [ ]:
import seaborn as sns
sns.boxplot(corpus_tokens_count)

In [ ]:
#30 most common tokens
corpus_tokens_count.head(50)

##### Mots peu significatifs

##### Mots uniques

In [ ]:
#30 last common tokens
corpus_tokens_count.tail(30)

In [ ]:
# définition d'une liste d'unique words --> mots présents une seule fois et pas très utiles
list_unique_words = corpus_tokens_count[corpus_tokens_count==1]
list_unique_words[:30]

In [ ]:
len(list_unique_words)

In [ ]:
# définition d'une liste d'unique words --> mots présents une seule fois et pas très utiles
list_words_10 = corpus_tokens_count[corpus_tokens_count<=10]
list_words_10[:30]

In [ ]:
len(list_words_10)

In [ ]:
len(corpus_tokens)

In [ ]:
#corpus_tokens = corpus_tokens

#list_words_10 = corpus_tokens_count[corpus_tokens_count<=10]

# Éliminer les mots de 'list_words_10' de 'corpus_tokens'
corpus_tokens = [word for word in corpus_tokens if word not in list_words_10]

len(corpus_tokens)


In [ ]:
#longuer de la liste de tokens uniques
len(set(corpus_tokens))

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize

# Fonction pour enlever les mots contenus dans list_words_10
def remove_words(text, words_to_remove):
    tokens = word_tokenize(text)
    cleaned_tokens = [token for token in tokens if token not in words_to_remove]
    cleaned_text = ' '.join(cleaned_tokens)
    return cleaned_text

# Appliquer la fonction pour enlever les mots de la liste list_words_10 à la colonne 'Processed_Text'
df_1['Processed_Text'] = df_1['Processed_Text'].apply(remove_words, args=(list_words_10,))



In [ ]:
df_1.tail()

In [ ]:
#Vérification
word_to_search = "sickness"

# Utiliser str.contains pour filtrer les lignes contenant le mot recherché
result = df_1[df_1['Processed_Text'].str.contains(word_to_search)]

# Afficher le résultat
print(result)


In [ ]:
#Drop de la phrase en alphabeth non-occidental  et insulte

# Import de Pandas
import pandas as pd

# Identification des lignes contenant le mot d'insulte
rows_to_drop = df_1['Original_Text'].str.contains('bitch', case=False)

# Suppression des lignes correspondantes du DataFrame
df_1 = df_1[~rows_to_drop]


In [ ]:
import random

# Calculer la taille de l'échantillon (20% de la taille originale)
sample_size = int(0.2 * len(corpus_tokens))

# Échantillonner 20% des tokens de la liste
sampled_tokens = random.sample(corpus_tokens, sample_size)

# Afficher le résultat
#print(sampled_tokens)


### Associer chaque mot du corpus à un identifiant clé unique

In [ ]:
import nltk
import gensim
from gensim import corpora

# Tokenize chaque document dans la colonne 'Processed_Text'
corpus_tokens = df_1['Processed_Text'].apply(nltk.word_tokenize)

# Créer un dictionnaire
dictionary = corpora.Dictionary(corpus_tokens)
print(dictionary)

Le dictionnaire a été construit ici avec un total de 3940 termes uniques extraits des 19937 documents du corpus.

### Bigrammes, trigrammes et Ngrammes

#### Utilisation du module Phrases de Gesim pour anlayser les bigrammes et les trigrammes

In [ ]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser

nltk.download('punkt')

# Tokenize chaque document dans la colonne 'Processed_Text'
corpus_tokens = df_1['Processed_Text'].apply(nltk.word_tokenize).tolist()

# Calculer le nombre total de bigrams possibles dans le corpus
#total_bigrams = sum(len(tokens) - 1 for tokens in corpus_tokens)

# Définir un pourcentage minimum d'apparition des bigrams
#percentage_threshold = 0.80

# Calculer le seuil en fonction du pourcentage
#threshold = total_bigrams * percentage_threshold

# Créer un modèle Phrases avec le seuil
#bigram = Phrases(corpus_tokens, min_count=10, threshold=threshold)

# Créer un Phraser pour accélérer la recherche des bigrams
#bigram_phraser = Phraser(bigram)

# Appliquer le Phraser pour obtenir les textes avec les bigrams
#text_with_bigrams = [bigram_phraser[tokens] for tokens in corpus_tokens]

# Afficher les textes avec les bigrams
#for text in text_with_bigrams:
    #print(" ".join(text))


In [ ]:
from nltk.util import bigrams, trigrams, ngrams

# tokeniser le corpus pour en extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_bigrams = list(nltk.bigrams(corpus_tokens))

# affichage
print(corpus_bigrams[:10])

In [ ]:
import nltk
from nltk.util import bigrams
from nltk import FreqDist

# Tokeniser le corpus pour extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_bigrams = list(bigrams(corpus_tokens))

# Calculer la fréquence des bigrammes
freq_dist = FreqDist(corpus_bigrams)

# Obtenir les 10 bigrammes les plus fréquents
top_20_bigrams = freq_dist.most_common(20)

# Afficher les 20 bigrammes les plus fréquents
print("Les 20 bigrammes les plus fréquents sont :")
for bigram, frequency in top_20_bigrams:
    print(bigram, ":", frequency)


In [ ]:
import nltk
from nltk.util import bigrams
from nltk import FreqDist

# Tokeniser le corpus pour extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_bigrams = list(bigrams(corpus_tokens))

# Calculer la fréquence des bigrammes
freq_dist = FreqDist(corpus_bigrams)

# Calculer le nombre total de bigrammes
total_bigrams = len(corpus_bigrams)

# Calculer le pourcentage d'occurrence pour chaque bigramme
bigram_percentage = [(bigram, frequency / total_bigrams * 100) for bigram, frequency in freq_dist.items()]

# Trier les bigrammes par pourcentage décroissant
sorted_bigrams = sorted(bigram_percentage, key=lambda x: x[1], reverse=True)

# Obtenir les 10 bigrammes les plus fréquents
top_10_bigrams = sorted_bigrams[:10]

# Afficher les 10 bigrammes les plus fréquents avec leur pourcentage
print("Les 10 bigrammes les plus fréquents avec leur pourcentage sont :")
for bigram, percentage in top_10_bigrams:
    print(f"{bigram} : {percentage:.2f}%")


In [ ]:
! pip install gensim

In [ ]:
!pip install --upgrade gensim


In [ ]:
! pip install scipy

In [ ]:
#import gensim, spacy, logging, warnings
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(df_1['Processed_Text'], min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[(df_1['Processed_Text'])], threshold=100)

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
df_1.head()

##### Pondération des bigrammes

In [ ]:
import nltk
from nltk.util import bigrams
from nltk import FreqDist

# Tokeniser le corpus pour extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_bigrams = list(bigrams(corpus_tokens))

# Calculer la fréquence des bigrammes
freq_dist = FreqDist(corpus_bigrams)

# Obtenir le nombre total de bigrammes
total_bigrams = len(corpus_bigrams)

# Créer un dictionnaire pour stocker les bigrammes pondérés
weighted_bigrams = {}

# Calculer la fréquence relative de chaque bigramme
for bigram, frequency in freq_dist.items():
    weighted_bigrams[bigram] = frequency / total_bigrams

# Afficher les bigrammes pondérés
#for bigram, weight in weighted_bigrams.items():
    #print(bigram, ":", weight)

# Afficher les 10 premiers bigrammes pondérés
count = 0
for bigram, weight in weighted_bigrams.items():
    print(bigram, ":", weight)
    count += 1
    if count == 10:
        break


In [ ]:
# Création d'un bag of bigrams
corpus_bigrams = list(bigrams(corpus_tokens))

# Créer un dictionnaire de bigrammes et leurs fréquences
bigram_freq = FreqDist(corpus_bigrams)

# Afficher les 10 premiers bigrammes
for bigram, freq in bigram_freq.most_common(10):
    print(bigram, ":", freq)

In [ ]:
from nltk.util import bigrams, trigrams, ngrams

# tokeniser le corpus pour en extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_trigrams = list(nltk.trigrams(corpus_tokens))

# affichage
print(corpus_trigrams[:10])

In [ ]:
import nltk
from nltk.util import bigrams
from nltk import FreqDist

# Tokeniser le corpus pour extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_trigrams = list(trigrams(corpus_tokens))

# Calculer la fréquence des bigrammes
freq_dist_tr = FreqDist(corpus_trigrams)

# Obtenir les 20 trigrammes les plus fréquents
top_20_trigrams = freq_dist_tr.most_common(20)

# Afficher les 20 trigrammes les plus fréquents
print("Les 20 trigrammes les plus fréquents sont :")
for trigram, frequency in top_20_trigrams:
    print(trigram, ":", frequency)

In [ ]:
from nltk.util import bigrams, trigrams, ngrams

# tokeniser le corpus pour en extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_ngrams = list(nltk.ngrams(corpus_tokens, 5))

# affichage
print(corpus_ngrams[:10])

In [ ]:
import nltk
from nltk.util import bigrams
from nltk import FreqDist

# Tokeniser le corpus pour extraire les bigrammes
corpus_tokens = nltk.word_tokenize(raw_corpus)

# Créer les bigrammes
corpus_ngrams = list(nltk.ngrams(corpus_tokens, 5))

# Calculer la fréquence des bigrammes
freq_dist_tr = FreqDist(corpus_ngrams)

# Obtenir les 10 trigrammes les plus fréquents
top_10_ngrams = freq_dist_tr.most_common(10)

# Afficher les 10 ngrammes les plus fréquents
print("Les 10 ngrammes les plus fréquents sont :")
for ngram, frequency in top_10_ngrams:
    print(ngram, ":", frequency)

### **Traîtement du corpus**

#### Vectorisation avec One-Hot Encoding

In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Créer un vectoriseur de comptage (CountVectorizer)
vectorizer = CountVectorizer(decode_error='ignore', stop_words='english', lowercase=True,)

# Appliquer le vectoriseur sur le corpus prétraité
corpus_vectorized = vectorizer.fit_transform(df_1['Processed_Text'])
#corpus_vectorized = vectorizer.fit_transform(corpus_tokens)

# Obtenir les noms des caractéristiques (mots)
feature_names = vectorizer.get_feature_names_out()

# Affichez l'ensemble du vocabulaire
print("Ensemble du vocabulaire :", feature_names)

# Affichez la matrice vocabulaire-documents
print("\nMatrice vocabulaire-documents :\n", corpus_vectorized.toarray())


In [ ]:
import sklearn
from sklearn.feature_extraction.text import CountVectorizer

# Créer un vectoriseur de comptage (CountVectorizer)
vectorizer = CountVectorizer(decode_error='ignore', stop_words='english', lowercase=True,)

# Appliquer le vectoriseur sur le corpus prétraité
corpus_vectorized_1 = vectorizer.fit_transform(df_1['Processed_Text'])
corpus_vectorized_1

### Tranformation de la matrice creuse en matrice dense

In [ ]:
# Transformation matrice creuse en matrice dense
corpus_vectorized_dense = corpus_vectorized_1.todense()

#### Obtention des noms des caractéristiques (mots)

In [ ]:
# Obtenir les noms des caractéristiques (mots)
feature_names = vectorizer.get_feature_names_out()

In [ ]:
# voir les 20 premiers
feature_names[:20]

In [ ]:
# voir les 20 derniers
feature_names[-20:]

#### Bag of words : TF-IDF. Term Frequency-Inverse Document Frequency

Ce calcul évalue l’importance d’un terme contenu dans un document, relativement à une collection. Si un mot rare est particulièrement présent dans un document, son TF-IDF sera très élevé.
TF(t,i) = Nombre d’apparitions du terme t dans le document i / Nombre total de termes dans le document

IDF(t) = log(Nombre de documents dans le corpus / Nombre de documents où t apparaît)

D’où TF(t,i) * IDF(t) donne la pertinence de chaque document vis-à-vis d’un mot.


In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer une instance du vectoriseur TfidfVectorizer
vectorizer_Tf = TfidfVectorizer(decode_error='ignore', stop_words='english')

# Appliquer le vectoriseur sur la colonne 'Processed_Text'
corpus_vectorised = vectorizer_Tf.fit_transform(df_1['Processed_Text'])

# Convertir les caractéristiques (corpus_vectorised ici) en une représentation de matrice creuse
corpus_vectorised = corpus_vectorised.toarray()

# Afficher les dimensions de la matrice de caractéristiques
print("Dimensions de la matrice de caractéristiques :", corpus_vectorised.shape)

# Afficher l'ensemble du vocabulaire
print("Ensemble du vocabulaire :", vectorizer_Tf.get_feature_names_out())

# Afficher la matrice TF-IDF
print("\nMatrice TF-IDF \n", corpus_vectorised)


#### Processus detaillé

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer

# Créer une instance du vectoriseur TfidfVectorizer
vectorizer_Tf = TfidfVectorizer(decode_error='ignore', stop_words='english')

# Appliquer le vectoriseur sur la colonne 'Processed_Text'
corpus_vectorised = vectorizer_Tf.fit_transform(df_1['Processed_Text'])
corpus_vectorised

#### Transformation de la matrice éparse en matrice dense

In [ ]:
corpus_vectorised_d = corpus_vectorised.todense()
corpus_vectorised_d

In [ ]:
# Obtenir les noms des caractéristiques (mots)
feature_names_Tf = vectorizer_Tf.get_feature_names_out()

In [ ]:
# voir les 20 premiers
feature_names_Tf[:20]

In [ ]:
# voir les 20 derniers
feature_names_Tf[-20:]

In [ ]:
import gensim.downloader as api
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

dataset = api.load("text8")
dct = Dictionary(dataset)  # fit dictionary
corpus = [dct.doc2bow(line) for line in dataset]  # convert corpus to BoW format

model = TfidfModel(corpus)  # fit model
vector = model[corpus[0]]  # apply model to the first corpus document

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Concaténer tous les textes de la colonne 'Processed_text' en une seule chaîne
all_processed_text = ' '.join(df_1['Processed_Text'])

# Créer un objet WordCloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(all_processed_text)

# Afficher le nuage de mots
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


## LDA

#### Conversion des données

#### Appel d'un dictionnaire à partir des données préparées

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim.models import CoherenceModel

In [ ]:
# Tokenize chaque document dans la colonne 'Processed_Text'
corpus_tokens = df_1['Processed_Text'].apply(nltk.word_tokenize)

# Créer un dictionnaire
dictionary = gensim.corpora.Dictionary(corpus_tokens)
print(dictionary)

##### Filtrage du dictionnaire

In [ ]:
dictionary.filter_extremes(no_below=15, no_above=0.1, keep_n=1000)

Nous avons filtré le dictionnaire pour ne garder que les mots pouvant apparaitre dans un total d'entre 99 et 15 documents.

##### Transformation en format Bag OF Words (bow)

In [ ]:
bow_corpus = [dictionary.doc2bow(doc) for doc in corpus_tokens]

In [ ]:
# Imprimer le nombre de documents dans bow_corpus
print("Nombre de documents dans bow_corpus :", len(bow_corpus))

#### Exécution du LDA

On applique le topic modeling à l’aide de la fonction LdaMulticore de gensim en prenant bien soin de préciser le nombre de topic à extraire du corpus, le mapping entre les identifiants des mots (entiers) et les mots (chaîne de caractères) et le nombre à effectuer d’itération dans le corpus.

Le nombre de topic à extraire constitue une variable importante du modèle, son choix est souvent cruciale tout comme celui du nombres de classes en clustering par exemple.


In [ ]:
from gensim import corpora
from gensim.models import LdaMulticore

#Création du model LDA
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics = 4, id2word = dictionary, passes = 100)


In [ ]:
print(lda_model)

In [ ]:
topics = []
for idx, topic in lda_model.print_topics(-1) :
    print("Topic: {} -> Words: {}".format(idx, topic))
    topics.append(topic)

#### Cohérence du modèle

Les mesures de cohérence évaluent le degré de similitude sémantique entre les mots les mieux notés dans le topics. Ces mesures aident à faire la distinction entre les topics sémantiquement interprétables et les topics dû à des inférences statistiques.

Un bon modèle LDA comporte une cohérence comprise entre 0.4 et 0.7 (au-delà et en dessous le modèle est très probablement erroné). La cohérence pour un modèle LDA est calculée en procédant aux étapes suivantes :

Segmentation : création de paires de mots à partir de sous-ensembles ;
Calcul des probabilités : calcul probabilité d’occurrence d’un mot ;
Mesure de confirmation : vérification « dans quelle mesure » un sous-ensemble de mots supporte un autre sous-ensemble de mots dans chaque paire ;
Agrégation : agrégation de toutes les valeurs calculées à l’étape précédente en une seule valeur qui est notre score final de cohérence de sujet.

In [ ]:
from gensim.models import CoherenceModel
from gensim import corpora, models

coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus_tokens, dictionary=dictionary)
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

##### Stockage des résultats

In [ ]:
all_topic_model = []
for i in range(len(topics)):
  str = topics[i].split(' + ')
  topic_model = []
  for j in range(10):
    weight = str[j][0:5]
    word = str[j][7:len(str[j])-1]
    topic_model.append((weight, word))
  all_topic_model.append(topic_model)

In [ ]:
df_topic_model = pd.DataFrame(all_topic_model)
df_topic_model.rename(index = {0: "Topic 1", 1: "Topic 2", 2: "Topic 3", 3: "Topic 4", 4: "Topic 5", 5: "Topic 6", 6: "Topic 7", 7: "Topic 8", 8: "Topic 9", 9: "Topic 10"})

#### Visualisation des résultats


In [ ]:
!pip install pyLDAvis

In [ ]:
import pyLDAvis

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import pyLDAvis.gensim_models
from gensim.models import LdaMulticore
import pandas as pd

# Visualisation
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda_model, bow_corpus, dictionary)


## Identification du sujet le plus important

### Réduction de dimension avec TSNE et visualisation

In [ ]:
df_1.head()

In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Extraire les distributions de sujets pour chaque document
doc_topic_vectors = [lda_model[doc] for doc in bow_corpus]

# Créer une liste de dictionnaires pour chaque document
topic_data = []

for topics in doc_topic_vectors:
    cleaned_topics = {f"Topic {t[0]}": t[1] if not np.isnan(t[1]) else 0.0 for t in topics}
    topic_data.append(cleaned_topics)

# Convertir en DataFrame
df_topic_sents_keywords = pd.DataFrame(topic_data)

# Vérifier et remplacer les NaN dans l'ensemble du DataFrame
df_topic_sents_keywords = df_topic_sents_keywords.fillna(0.0)

# Appliquer t-SNE pour réduire la dimension à 2
tsne_model = TSNE(n_components=2, random_state=42)
tsne_lda = tsne_model.fit_transform(df_topic_sents_keywords)

# Créer un DataFrame pour les résultats t-SNE
tsne_df = pd.DataFrame(tsne_lda, columns=['tsne1', 'tsne2'])

# Visualiser les résultats avec seaborn
plt.figure(figsize=(10, 8))
sns.scatterplot(x='tsne1', y='tsne2', data=tsne_df)
plt.title('t-SNE Visualization of LDA Topics for Documents')
plt.show()





In [ ]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Extraire les distributions de sujets pour chaque document
doc_topic_vectors = [lda_model[doc] for doc in bow_corpus]

# Créer une liste de dictionnaires pour chaque document
topic_data = []

for topics in doc_topic_vectors:
    cleaned_topics = {f"Topic {t[0]}": t[1] if not np.isnan(t[1]) else 0.0 for t in topics}
    topic_data.append(cleaned_topics)

# Convertir en DataFrame
df_topic_sents_keywords = pd.DataFrame(topic_data)

# Vérifier et remplacer les NaN dans l'ensemble du DataFrame
df_topic_sents_keywords = df_topic_sents_keywords.fillna(0.0)

# Appliquer t-SNE pour réduire la dimension à 2
tsne_model = TSNE(n_components=2, random_state=42)
tsne_lda = tsne_model.fit_transform(df_topic_sents_keywords)

# Créer un DataFrame pour les résultats t-SNE
tsne_df = pd.DataFrame(tsne_lda, columns=['tsne1', 'tsne2'])

# Ajouter une colonne pour le nom du topic principal (Top 1)
tsne_df['Top_Topic'] = df_topic_sents_keywords.idxmax(axis=1)

# Ajouter une colonne pour la couleur du topic principal
topic_colors = {
    'Topic 0': 'red',
    'Topic 1': 'green',
    'Topic 2': 'blue',
    'Topic 3': 'purple'
}

tsne_df['Color'] = tsne_df['Top_Topic'].map(topic_colors)

# Visualiser les résultats avec seaborn
plt.figure(figsize=(10, 8))
sns.scatterplot(x='tsne1', y='tsne2', hue='Color', data=tsne_df, palette=topic_colors.values(), legend='full')

# Ajouter des annotations pour les points
#for idx, row in tsne_df.iterrows():
    #plt.annotate(row['Top_Topic'], (row['tsne1'], row['tsne2']), fontsize=8)

plt.title('t-SNE Visualization of LDA Topics for Documents')
plt.show()


## Analyse de sentiments avec TextBlob

In [ ]:
# import de la librairie
! pip install TextBlob


In [ ]:
import textblob

from textblob import TextBlob

# Calcule de la polarité
df_1['polarity'] = df_1['Processed_Text'].apply(lambda x: TextBlob(x).sentiment[0])
df_1['subjectivity'] = df_1['Processed_Text'].apply(lambda x: TextBlob(x).sentiment[1])

# Définir les étiquettes de sentiment en fonction de la polarité
df_1['Sentiment'] = df_1['polarity'].apply(lambda x: 'Positif' if x > 0 else ('Négatif' if x < 0 else 'Neutre'))
df_1.head(20)